In [1]:
# Have to patch sys.path until we have tighter taucmdr/anaconda integration.
# Need to move towards taucmdr as a conda package.
import os
import sys
try:
    import taucmdr
except ImportError:
    sys.path.insert(0, os.path.join(os.environ['__TAUCMDR_HOME__'], 'packages'))
finally:
    from taucmdr.model.project import Project

In [2]:
%pylab
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import pandas as pd
import math
import numpy as np
import operator

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
num_trials = Project.selected().experiment().num_trials
trials = Project.selected().experiment().trials(xrange(0, num_trials))
trial_data = {}
for i in xrange(0, num_trials):
    trial_data[i] = trials[i].get_data()

In [4]:
#trial0_intervals = trial0_data[0][0][0].interval_data()
#trial0_intervals.plot(kind='bar')
#plt.show()

In [5]:
trial_intervals = {}
for trial in xrange(0, num_trials):
    trial_intervals[trial] = {}
    for i in xrange(0, len(trial_data[trial])):
        trial_intervals[trial][i] = {}
        for j in xrange(0, len(trial_data[trial][i])):
            trial_intervals[trial][i][j] = {}
            for k in xrange(0, len(trial_data[trial][i][j])):
                trial_intervals[trial][i][j][k] = trial_data[trial][i][j][k].interval_data()

In [6]:
from taucmdr.error import InternalError
def get_dfs_depth(dfs, level=1):
    depth = 0
    if not isinstance(dfs, dict) or not dfs:
        return level
    return max(get_dfs_depth(dfs[k], level + 1) for k in dfs)

def get_regions(dfs):
    regions = set()
    depth = get_dfs_depth(dfs)
    
    # Contains profile data for 1 thread
    if depth == 1:
        regions.update(dfs.index.tolist())
    # Contains profile data for 1 trial with multiple threads
    elif depth == 4:
        for rank in dfs:
            for context in dfs[rank]:
                for thread in dfs[rank][context]:
                    regions.update(dfs[rank][context][thread].index.tolist())
    # Contains profile data for an experiment with multiple trials
    elif depth == 5:
        for tr in dfs:
            for rank in dfs[tr]:
                for context in dfs[tr][rank]:
                    for thread in dfs[tr][rank][context]:
                        regions.update(dfs[tr][rank][context][thread].index.tolist())
    # Otherwise, not data is not valid
    else:
        raise InternalError('Invalid experiment data')
    return regions

def largest_stddev(dfs,n):
    # Use Welford's Method (see https://stackoverflow.com/questions/895929/how-do-i-determine-the-standard-deviation-stddev-of-a-set-of-values)
    if get_dfs_depth(dfs) != 4:
        raise InternalError('Invalid experiment data. Must pass 1 trial only.')
    regions = get_regions(dfs)
    stddevs = {}
    for region in regions:
        mean = 0.0
        stddev = 0.0
        k = 1
        for rank in dfs:
            for context in dfs[rank]:
                for thread in dfs[rank][context]:
                    try:
                        exclusive_time = dfs[rank][context][thread].at[region, 'Exclusive']
                    except KeyError:
                        pass
                    else:
                        tmpM = mean
                        mean += (exclusive_time - tmpM) / k
                        stddev += (exclusive_time - tmpM) * (exclusive_time - mean)
                        k += 1

        stddev = math.sqrt(stddev/(k-1))
        stddevs[region] = stddev

        
    return [x for x in sorted(stddevs.iteritems(), key=operator.itemgetter(1), reverse=True)[:n]]


def largest_correlation(dfs,n):
    if get_dfs_depth(dfs) != 5:
        raise InternalError('Invalid experiment data. Must pass multiple trials.')
    regions = get_regions(dfs)
    regions.remove('.TAU application')
    regions_dict = {}
    for region in regions:
        regions_dict[region] = []
        for tr in dfs:
            for rank in dfs[tr]:
                for context in dfs[tr][rank]:
                    for thread in dfs[tr][rank][context]:
                        try:
                            regions_dict[region].append(dfs[tr][rank][context][thread].at[region, 'Exclusive'])
                        except:
                            regions_dict[region].append(0)
    regions_dict['.TAU application'] = []
    for tr in dfs:
        for rank in dfs[tr]:
            for context in dfs[tr][rank]:
                for thread in dfs[tr][rank][context]:
                    regions_dict['.TAU application'].append(dfs[tr][rank][context][thread].at['.TAU application', 'Inclusive'])

    corr_coef = {}
    for region in regions:
        corr_coef[region] = np.corrcoef(regions_dict[region], regions_dict['.TAU application'])[0,1]
    

    return [x for x in sorted(corr_coef.iteritems(), key=operator.itemgetter(1), reverse=True)[:n]]

def largest_exclusive(df,n):
    rows = df.nlargest(n, 'Exclusive')
    return list(rows.index)

def hotspots(dfs, n, flag):
    if flag == 0:
        largest = largest_exclusive(dfs,n)
    elif flag == 1:
        largest = largest_stddev(dfs,n)
    elif flag == 2:
        largest = largest_correlation(dfs,n)
    else:
        print 'Invalid flag'
    y = ['exclusive time', 'standard deviation', 'correlation to total runtime']
    print 'Hotspot Analysis Summary'
    print '='*80
    print 'The code regions with largest %s are: ' %y[flag]
    for i in xrange(0,n):
        print '%s: %s' %(i+1, largest[i])

hotspots(trial_intervals[0][0][0][0], 5, 0)
hotspots(trial_intervals[0], 5, 1)
hotspots(trial_intervals, 5, 2)

Hotspot Analysis Summary
The code regions with largest exclusive time are: 
1: void compute(double **, double **, double **, int, int, int) C [{matmult.c} {90,1}-{109,1}]  
2: .TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}]   => double do_work(void) C [{matmult.c} {132,1}-{168,1}]   => void compute(double **, double **, double **, int, int, int) C [{matmult.c} {90,1}-{109,1}]  
3: .TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}]   => double do_work(void) C [{matmult.c} {132,1}-{168,1}]   => void compute_interchange(double **, double **, double **, int, int, int) C [{matmult.c} {111,1}-{130,1}]  
4: void compute_interchange(double **, double **, double **, int, int, int) C [{matmult.c} {111,1}-{130,1}]  
5: double multiply(double, double) C [{matmult.c} {59,1}-{61,1}] [THROTTLED]
Hotspot Analysis Summary
The code regions with largest standard deviation are: 
1: ('.TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1